# EMG分类器详解与对比

## 教学内容

### 监督学习分类器
1. **SVM** (Support Vector Machine) - 支持向量机
2. **KNN** (K-Nearest Neighbors) - K近邻
3. **Random Forest** - 随机森林
4. **简单神经网络** (MLP)

### 无监督学习
5. **K-Means聚类**
6. **层次聚类**

### 对比分析
- 准确率对比
- 训练时间对比
- 参数敏感性
- 适用场景

In [ ]:
# ===== 第一步：导入所有需要的Python库 =====

import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import time

# ===== 第二步：添加项目路径（多重后备方案） =====
project_root = Path.cwd().parent
if not (project_root / 'code').exists():
    project_root = Path.cwd()

print(f"项目根目录: {project_root}")

# 清理旧路径
sys.path = [p for p in sys.path if 'EMG_Learning_Project' not in p]
sys.path.insert(0, str(project_root))

# ===== 第三步：导入自定义模块（三种方法） =====
import importlib

try:
    from code.week07_feature_extraction.features import EMGFeatures
    from code.week06_preprocessing.filters import EMGFilters
    print("✓ 模块导入成功！")
    
except ImportError as e1:
    print(f"方法1失败，尝试方法2...")
    try:
        features_module = importlib.import_module('code.week07_feature_extraction.features')
        filters_module = importlib.import_module('code.week06_preprocessing.filters')
        
        EMGFeatures = features_module.EMGFeatures
        EMGFilters = filters_module.EMGFilters
        print("✓ 模块导入成功（方法2）！")
        
    except Exception as e2:
        print(f"方法2失败，尝试方法3...")
        sys.path.insert(0, str(project_root / 'code' / 'week07_feature_extraction'))
        sys.path.insert(0, str(project_root / 'code' / 'week06_preprocessing'))
        
        from features import EMGFeatures
        from filters import EMGFilters
        print("✓ 模块导入成功（方法3）！")

# ===== 第四步：配置中文显示 =====
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

print("✓ 导入成功！")

## 准备数据集

生成三种手势的EMG数据用于分类。

In [ ]:
# ===== 生成三种手势的EMG数据 =====

def generate_gesture_data(gesture, n_samples=30):
    """
    生成指定手势的模拟EMG数据
    
    参数:
        gesture: 手势类型 ('rest', 'fist', 'open')
        n_samples: 生成样本数量（默认30个）
    
    返回:
        samples: 特征字典列表，每个样本包含18个特征 + 手势标签
    """
    # --- 初始化参数 ---
    fs = 1000  # 采样率：1000 Hz
    filters = EMGFilters(fs=fs)  # 创建滤波器对象
    samples = []  # 存储所有样本的列表
    
    # --- 循环生成n_samples个样本 ---
    for _ in range(n_samples):
        # 1. 生成时间轴（2秒信号）
        t = np.linspace(0, 2, 2000)  # 0到2秒，共2000个点
        
        # 2. 生成基础噪声（静息状态）
        signal = np.random.normal(0, 0.02, len(t))
        # 均值0，标准差0.02的高斯噪声
        
        # 3. 根据手势类型添加特定的信号成分
        if gesture == 'rest':
            # 【静息】只保留噪声，不添加额外信号
            pass  # 什么都不做
            
        elif gesture == 'fist':
            # 【握拳】强收缩，高频率（70-140 Hz）
            for freq in range(70, 140, 15):  # 频率：70, 85, 100, 115, 130 Hz
                signal += 0.5 * np.sin(2 * np.pi * freq * t)
                # 振幅0.5 mV - 代表强有力的握拳动作
                
        elif gesture == 'open':
            # 【张开】中等收缩，中等频率（60-120 Hz）
            for freq in range(60, 120, 15):  # 频率：60, 75, 90, 105 Hz
                signal += 0.3 * np.sin(2 * np.pi * freq * t)
                # 振幅0.3 mV - 代表中等力度的张开动作
        
        # 4. 预处理信号（去噪）
        signal_clean = filters.preprocess_emg(signal, remove_powerline=True)
        # 应用带通滤波和陷波滤波
        
        # 5. 提取时域特征（10种）
        time_feat = EMGFeatures.extract_time_features(signal_clean)
        
        # 6. 提取频域特征（8种）
        freq_feat = EMGFeatures.extract_freq_features(signal_clean, fs=fs)
        
        # 7. 合并特征并添加手势标签
        sample = {**time_feat, **freq_feat, 'gesture': gesture}
        # 使用字典解包合并时域、频域特征，并添加手势标签
        
        samples.append(sample)
    
    return samples

# ===== 生成所有手势数据 =====
data = []  # 存储所有数据
gestures = ['rest', 'fist', 'open']  # 三种手势类型

print("开始生成数据...")
for gesture in gestures:
    # 为每种手势生成30个样本
    gesture_data = generate_gesture_data(gesture, n_samples=30)
    data.extend(gesture_data)  # 添加到总数据列表
    print(f"  ✓ {gesture}: 30个样本")

# 转换为pandas DataFrame
df = pd.DataFrame(data)

print(f"\n✓ 数据生成完成：{len(df)}个样本")
print(f"  每种手势: 30个样本")
print(f"  总特征数: {df.shape[1] - 1}个（不含标签）")  # -1是因为'gesture'是标签列

In [ ]:
# ===== 准备训练数据 =====

# --- 第一步：分离特征和标签 ---
X = df.drop('gesture', axis=1).values
# drop('gesture', axis=1): 删除'gesture'列（标签列），保留所有特征列
# .values: 转换为numpy数组
# X的形状: (90, 18) - 90个样本，每个18个特征

y = df['gesture'].values
# 提取标签列：'rest', 'fist', 'open'
# y的形状: (90,) - 90个文本标签

# --- 第二步：标签编码（文本转数字） ---
# 机器学习算法需要数字标签，不能直接处理文本
label_map = {'rest': 0, 'fist': 1, 'open': 2}
# 定义映射规则：rest→0, fist→1, open→2

y_numeric = np.array([label_map[g] for g in y])
# 使用列表推导式批量转换
# 遍历y中的每个标签g，通过label_map转换为数字

# --- 第三步：划分训练集和测试集 ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y_numeric,           # 输入：特征和标签
    test_size=0.3,          # 测试集占30%（27个样本）
    random_state=42,        # 随机种子，保证结果可重复
    stratify=y_numeric      # 分层采样，保证每个类别在训练集和测试集中的比例相同
)
# 训练集: 70% × 90 = 63个样本
# 测试集: 30% × 90 = 27个样本

# --- 第四步：特征归一化（标准化） ---
# 为什么需要归一化？
# - 不同特征的数值范围可能差异很大（如MAV可能是0.1，而WL可能是100）
# - 某些算法（如SVM、KNN）对特征尺度敏感
# - 归一化后所有特征都在相似的数值范围内，训练更稳定

scaler = StandardScaler()
# StandardScaler: 标准化方法，将每个特征转换为均值=0，标准差=1
# 公式: z = (x - mean) / std

X_train_scaled = scaler.fit_transform(X_train)
# fit_transform: 先计算训练集的均值和标准差，然后进行转换
# 必须用训练集的统计量！

X_test_scaled = scaler.transform(X_test)
# transform: 使用训练集的均值和标准差来转换测试集
# 注意：不能用fit_transform，避免数据泄露

# --- 第五步：打印数据信息 ---
print(f"训练集: {X_train.shape}")
print(f"  样本数: {X_train.shape[0]}")
print(f"  特征数: {X_train.shape[1]}")
print(f"\n测试集: {X_test.shape}")
print(f"  样本数: {X_test.shape[0]}")
print(f"  特征数: {X_test.shape[1]}")
print(f"\n标签分布:")
for gesture, code in label_map.items():
    train_count = np.sum(y_train == code)
    test_count = np.sum(y_test == code)
    print(f"  {gesture} (编码{code}): 训练集{train_count}个, 测试集{test_count}个")

---
## 1. SVM (支持向量机)

### 原理
- 寻找最优超平面分隔不同类别
- 使用核函数处理非线性问题
- 只关注边界附近的样本（支持向量）

### 优点
- 高维数据表现好
- 泛化能力强
- 理论基础扎实

### 缺点
- 训练较慢（大数据集）
- 参数调优复杂
- 难以解释

In [ ]:
# ===== 训练SVM（支持向量机）分类器 =====

print("训练SVM...")
print("=" * 50)

# --- 记录开始时间 ---
start_time = time.time()

# --- 第一步：创建SVM分类器对象 ---
svm = SVC(
    kernel='rbf',      # 核函数：径向基函数（Radial Basis Function）
                       # RBF能处理非线性问题，是最常用的核函数
    
    C=1.0,             # 正则化参数：控制分类边界的"软硬"程度
                       # C越大，边界越"硬"（容错率低，可能过拟合）
                       # C越小，边界越"软"（容错率高，可能欠拟合）
    
    gamma='scale',     # RBF核的参数：控制单个样本的影响范围
                       # 'scale': 自动计算 = 1 / (n_features * X.var())
    
    random_state=42    # 随机种子：保证结果可重复
)

# --- 第二步：训练模型 ---
svm.fit(X_train_scaled, y_train)
# fit(): 使用训练数据训练模型
# 输入：归一化后的特征矩阵 + 标签
# SVM会寻找最优的分类超平面

# --- 第三步：在测试集上预测 ---
y_pred_svm = svm.predict(X_test_scaled)
# predict(): 对测试集进行预测
# 输入：归一化后的测试特征
# 输出：预测的类别标签（0, 1, 2）

# --- 第四步：计算准确率 ---
acc_svm = accuracy_score(y_test, y_pred_svm)
# accuracy_score(): 计算预测准确率
# 准确率 = 预测正确的样本数 / 总样本数

# --- 第五步：计算训练时间 ---
train_time_svm = time.time() - start_time

# --- 第六步：打印结果 ---
print(f"  ✓ 训练完成")
print(f"  准确率: {acc_svm:.2%}")
print(f"  训练时间: {train_time_svm:.4f}秒")
print(f"  支持向量数: {len(svm.support_)}")
# 支持向量：决定分类边界的关键样本点
# 数量越少，模型越简洁

print("\n💡 SVM工作原理：")
print("  1. 在特征空间中寻找最优分类超平面")
print("  2. 最大化不同类别之间的间隔（margin）")
print("  3. 只有边界附近的样本（支持向量）影响决策")
print("  4. 使用核技巧处理非线性问题")

---
## 2. KNN (K近邻)

### 原理
- 找到最近的k个邻居
- 通过投票决定类别
- 简单直观，无需训练

### 优点
- 实现简单
- 无需训练过程
- 对异常值鲁棒

### 缺点
- 预测速度慢
- 需要存储所有训练数据
- 对特征缩放敏感

In [ ]:
# ===== 训练KNN（K近邻）分类器 =====

print("训练KNN...")
print("=" * 50)

# --- 记录开始时间 ---
start_time = time.time()

# --- 第一步：创建KNN分类器对象 ---
knn = KNeighborsClassifier(
    n_neighbors=5      # k值：考虑最近的5个邻居
                       # k越小：决策边界越复杂，可能过拟合
                       # k越大：决策边界越平滑，可能欠拟合
                       # 通常选择奇数，避免投票平局
)

# --- 第二步：训练模型 ---
knn.fit(X_train_scaled, y_train)
# fit(): "训练"模型（实际上只是存储训练数据）
# KNN是懒惰学习（Lazy Learning），训练阶段几乎不做计算
# 主要计算都在预测阶段

# --- 第三步：在测试集上预测 ---
y_pred_knn = knn.predict(X_test_scaled)
# predict(): 对每个测试样本：
#   1. 计算到所有训练样本的距离（欧氏距离）
#   2. 找出最近的k个邻居
#   3. 通过投票决定类别（多数表决）

# --- 第四步：计算准确率 ---
acc_knn = accuracy_score(y_test, y_pred_knn)

# --- 第五步：计算训练时间 ---
train_time_knn = time.time() - start_time

# --- 第六步：打印结果 ---
print(f"  ✓ 训练完成")
print(f"  准确率: {acc_knn:.2%}")
print(f"  训练时间: {train_time_knn:.4f}秒")
print(f"  k值: 5")

print("\n💡 KNN工作原理：")
print("  1. 对于新样本，找到训练集中最近的k个邻居")
print("  2. 这k个邻居进行投票")
print("  3. 票数最多的类别就是预测结果")
print("  4. 简单直观，但预测速度慢（需要遍历所有训练样本）")
print("\n⚠️  注意：")
print("  - KNN对特征尺度敏感，必须归一化")
print("  - 训练快，但预测慢（与训练集大小成正比）")
print("  - 需要存储所有训练数据，内存占用大")

---
## 3. Random Forest (随机森林)

### 原理
- 训练多个决策树
- 每棵树使用随机的特征子集
- 投票决定最终结果

### 优点
- 准确率高
- 不易过拟合
- 可以评估特征重要性
- 对参数不敏感

### 缺点
- 模型较大
- 可解释性一般

In [ ]:
# ===== 训练Random Forest（随机森林）分类器 =====

print("训练Random Forest...")
print("=" * 50)

# --- 记录开始时间 ---
start_time = time.time()

# --- 第一步：创建Random Forest分类器对象 ---
rf = RandomForestClassifier(
    n_estimators=100,   # 树的数量：使用100棵决策树
                        # 树越多，性能越好（但训练更慢）
                        # 通常100-500棵树是个好选择
    
    random_state=42     # 随机种子：保证结果可重复
)

# --- 第二步：训练模型 ---
rf.fit(X_train_scaled, y_train)
# fit(): 训练所有决策树
# 每棵树的训练过程：
#   1. 随机抽取训练样本（有放回抽样，bootstrap）
#   2. 随机选择特征子集
#   3. 构建决策树
# 这种随机性使模型更稳定，不易过拟合

# --- 第三步：在测试集上预测 ---
y_pred_rf = rf.predict(X_test_scaled)
# predict(): 所有树都对测试样本进行预测，然后投票
# 多数表决：哪个类别的票数最多，就预测为哪个类别

# --- 第四步：计算准确率 ---
acc_rf = accuracy_score(y_test, y_pred_rf)

# --- 第五步：计算训练时间 ---
train_time_rf = time.time() - start_time

# --- 第六步：打印结果 ---
print(f"  ✓ 训练完成")
print(f"  准确率: {acc_rf:.2%}")
print(f"  训练时间: {train_time_rf:.4f}秒")
print(f"  树的数量: 100")

print("\n💡 Random Forest工作原理：")
print("  1. 训练多棵决策树（集成学习）")
print("  2. 每棵树使用不同的训练样本和特征子集")
print("  3. 预测时所有树投票，多数表决")
print("  4. '三个臭皮匠，顶个诸葛亮' - 集体智慧")
print("\n✅ Random Forest优点：")
print("  - 准确率通常很高")
print("  - 不容易过拟合")
print("  - 可以评估特征重要性")
print("  - 对参数不敏感，容易使用")
print("\n⚠️  Random Forest缺点：")
print("  - 模型文件较大（需要存储所有树）")
print("  - 可解释性不如单个决策树")

---
## 4. MLP神经网络（最简单的深度学习）

### 网络结构
```
输入层 (18个特征)
    ↓
隐藏层1 (64个神经元) + ReLU
    ↓
隐藏层2 (32个神经元) + ReLU
    ↓
输出层 (3个类别) + Softmax
```

### 优点
- 可以学习复杂的非线性关系
- 适合大数据集
- 可扩展性强

### 缺点
- 需要更多训练数据
- 训练时间较长
- 超参数多

In [ ]:
# ===== 训练MLP神经网络（最简单的深度学习） =====

print("训练MLP神经网络...")
print("=" * 50)

# --- 记录开始时间 ---
start_time = time.time()

# --- 第一步：创建MLP分类器对象 ---
mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),  # 隐藏层结构：两个隐藏层，分别有64和32个神经元
                                   # (64, 32) 表示：输入层 → 64神经元 → 32神经元 → 输出层
    
    activation='relu',             # 激活函数：ReLU (Rectified Linear Unit)
                                   # ReLU(x) = max(0, x)，解决梯度消失问题
    
    max_iter=1000,                 # 最大迭代次数：最多训练1000轮
                                   # 如果提前收敛会自动停止
    
    random_state=42                # 随机种子：保证结果可重复
)

# --- 第二步：训练模型 ---
mlp.fit(X_train_scaled, y_train)
# fit(): 通过反向传播算法训练神经网络
# 训练过程：
#   1. 前向传播：输入 → 隐藏层1 → 隐藏层2 → 输出
#   2. 计算损失（预测值与真实值的差异）
#   3. 反向传播：计算梯度
#   4. 更新权重：使用梯度下降优化
#   5. 重复1-4步，直到收敛或达到最大迭代次数

# --- 第三步：在测试集上预测 ---
y_pred_mlp = mlp.predict(X_test_scaled)

# --- 第四步：计算准确率 ---
acc_mlp = accuracy_score(y_test, y_pred_mlp)

# --- 第五步：计算训练时间 ---
train_time_mlp = time.time() - start_time

# --- 第六步：打印结果 ---
print(f"  ✓ 训练完成")
print(f"  准确率: {acc_mlp:.2%}")
print(f"  训练时间: {train_time_mlp:.4f}秒")
print(f"  网络结构: 输入{X_train.shape[1]} → 64 → 32 → 输出3")
print(f"  实际迭代次数: {mlp.n_iter_}")
# n_iter_: 实际训练的轮数（可能小于max_iter）

print("\n💡 MLP（多层感知机）工作原理：")
print("  1. 多层神经元网络，模拟大脑神经元连接")
print("  2. 每个神经元执行：y = activation(w·x + b)")
print("     - w: 权重（通过训练学习）")
print("     - x: 输入")
print("     - b: 偏置")
print("     - activation: 激活函数（如ReLU）")
print("  3. 通过反向传播算法学习最优权重")
print("\n🧠 网络结构详解：")
print("  输入层(18) → [MAV, RMS, WL, ... 18个特征]")
print("       ↓")
print("  隐藏层1(64) → 64个神经元提取高级特征")
print("       ↓")
print("  隐藏层2(32) → 32个神经元进一步抽象")
print("       ↓")
print("  输出层(3) → [rest, fist, open] 的概率")
print("\n✅ MLP优点：")
print("  - 可以学习复杂的非线性关系")
print("  - 通用性强，适用于各种任务")
print("  - 可扩展到更深的网络（深度学习）")
print("\n⚠️  MLP缺点：")
print("  - 需要较多训练数据")
print("  - 训练时间较长")
print("  - 需要调整多个超参数（层数、神经元数、学习率等）")
print("  - 可解释性差（'黑盒'模型）")

---
## 监督学习分类器对比

In [ ]:
# ===== 汇总监督学习分类器结果 =====

# --- 第一步：创建结果汇总表 ---
results = pd.DataFrame([
    {
        '分类器': 'SVM', 
        '准确率': acc_svm * 100,           # 转换为百分比
        '训练时间(秒)': train_time_svm
    },
    {
        '分类器': 'KNN', 
        '准确率': acc_knn * 100, 
        '训练时间(秒)': train_time_knn
    },
    {
        '分类器': 'Random Forest', 
        '准确率': acc_rf * 100, 
        '训练时间(秒)': train_time_rf
    },
    {
        '分类器': 'MLP神经网络', 
        '准确率': acc_mlp * 100, 
        '训练时间(秒)': train_time_mlp
    }
])

# --- 第二步：打印对比表格 ---
print("\n分类器性能对比：")
print("=" * 60)
print(results.to_string(index=False))  # 不显示行索引
print("=" * 60)

# --- 第三步：创建可视化对比图 ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
# 创建1行2列的子图：左边是准确率，右边是训练时间

# 【左图】准确率对比
colors = ['#3498db', '#e74c3c', '#2ecc71', '#9b59b6']  # 蓝、红、绿、紫
bars1 = ax1.bar(results['分类器'], results['准确率'], color=colors, alpha=0.8)

ax1.set_ylabel('准确率 (%)', fontsize=12)
ax1.set_title('准确率对比', fontsize=13, fontweight='bold')
ax1.set_ylim(0, 100)  # y轴范围：0-100%
ax1.grid(True, alpha=0.3, axis='y')  # 水平网格线

# 在柱子顶部标注数值
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}%',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

# 【右图】训练时间对比
bars2 = ax2.bar(results['分类器'], results['训练时间(秒)'], color=colors, alpha=0.8)

ax2.set_ylabel('训练时间 (秒)', fontsize=12)
ax2.set_title('训练时间对比', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

# 在柱子顶部标注数值
for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.3f}s',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

# --- 第四步：性能分析 ---
print("\n📊 性能分析：")
print("-" * 60)

# 找出最佳分类器
best_acc_idx = results['准确率'].idxmax()
best_clf = results.loc[best_acc_idx, '分类器']
best_acc = results.loc[best_acc_idx, '准确率']

print(f"🏆 最高准确率: {best_clf} ({best_acc:.2f}%)")

# 找出最快的分类器
fastest_idx = results['训练时间(秒)'].idxmin()
fastest_clf = results.loc[fastest_idx, '分类器']
fastest_time = results.loc[fastest_idx, '训练时间(秒)']

print(f"⚡ 训练最快: {fastest_clf} ({fastest_time:.4f}秒)")

print("\n💡 选择建议：")
print("  - 如果追求最高准确率 → 选择Random Forest")
print("  - 如果需要快速训练 → 选择KNN")
print("  - 如果需要可解释性 → 选择决策树（未在此演示）")
print("  - 如果数据量大、问题复杂 → 选择神经网络")

---
## 5. K-Means聚类（无监督学习）

### 什么是聚类？
- **不需要标签**的机器学习
- 自动发现数据中的**群组**
- 用于**探索性数据分析**

### K-Means原理
1. 随机初始化k个中心点
2. 将每个样本分配到最近的中心
3. 更新中心点位置
4. 重复2-3直到收敛

In [ ]:
# ===== K-Means聚类（无监督学习） =====

print("执行K-Means聚类...")
print("=" * 50)

# --- 第一步：创建K-Means聚类对象 ---
kmeans = KMeans(
    n_clusters=3,       # 簇的数量：我们知道有3种手势，所以设为3
                        # 在真实应用中，可能需要通过肘部法则确定最优k值
    
    random_state=42     # 随机种子：保证结果可重复
)

# --- 第二步：执行聚类 ---
clusters = kmeans.fit_predict(X_train_scaled)
# fit_predict(): 训练并预测
# 聚类过程：
#   1. 随机初始化3个聚类中心
#   2. 将每个样本分配到最近的聚类中心
#   3. 重新计算每个簇的中心（簇内所有样本的平均值）
#   4. 重复步骤2-3，直到收敛（中心不再变化）

# --- 第三步：打印聚类结果 ---
print(f"  ✓ 聚类完成")
print(f"  聚类数: 3")
print(f"  迭代次数: {kmeans.n_iter_}")
# n_iter_: 算法收敛所需的迭代次数

print(f"\n各簇样本数:")
unique, counts = np.unique(clusters, return_counts=True)
for cluster_id, count in zip(unique, counts):
    print(f"  簇{cluster_id}: {count}个样本")

print(f"\n💡 什么是聚类（Clustering）？")
print("  - 聚类是无监督学习，不需要标签数据")
print("  - 自动发现数据中的内在分组结构")
print("  - 将相似的样本归为一类")
print("\n💡 K-Means算法原理：")
print("  1. 随机选择k个初始中心点")
print("  2. 计算每个样本到各中心的距离")
print("  3. 将样本分配到最近的中心（形成簇）")
print("  4. 重新计算每个簇的中心（取平均值）")
print("  5. 重复2-4步，直到中心不再变化")
print("\n⚠️  重要提示：")
print("  - 聚类是无监督的，簇的编号（0,1,2）不一定对应真实标签")
print("  - 簇0可能对应'fist'，也可能对应'rest'，需要人工解释")
print("  - 聚类用于探索数据结构，不能直接用于分类任务")

# --- 第四步：尝试将聚类结果与真实标签对应 ---
print("\n聚类结果与真实标签对应关系：")
for cluster_id in unique:
    # 找出该簇中的所有样本
    cluster_mask = (clusters == cluster_id)
    cluster_labels = y_train[cluster_mask]
    
    # 统计该簇中每种手势的数量
    label_counts = {}
    for label_code in [0, 1, 2]:
        count = np.sum(cluster_labels == label_code)
        if count > 0:
            gesture_name = list(label_map.keys())[list(label_map.values()).index(label_code)]
            label_counts[gesture_name] = count
    
    print(f"  簇{cluster_id}: {label_counts}")

In [ ]:
# ===== 使用PCA降维可视化聚类结果 =====

# 为什么需要PCA？
# - 我们的数据有18个特征（18维空间）
# - 无法直接在纸上或屏幕上画出18维的图
# - 使用PCA将18维降到2维，便于可视化

from sklearn.decomposition import PCA

# --- 第一步：PCA降维 ---
pca = PCA(n_components=2)  # 降到2维
X_pca = pca.fit_transform(X_train_scaled)
# PCA会找到数据方差最大的两个方向（主成分）
# 尽可能保留原始数据的信息

print(f"PCA降维完成")
print(f"  原始维度: 18维")
print(f"  降维后: 2维")
print(f"  保留的信息量: {pca.explained_variance_ratio_.sum():.2%}")
# explained_variance_ratio_: 每个主成分保留的方差比例

# --- 第二步：创建对比可视化 ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 【左图】真实标签
scatter1 = ax1.scatter(
    X_pca[:, 0],        # x轴：第一主成分
    X_pca[:, 1],        # y轴：第二主成分
    c=y_train,          # 颜色：真实标签（0,1,2）
    cmap='viridis',     # 颜色映射：黄绿蓝渐变
    alpha=0.6,          # 透明度60%
    s=50,               # 点的大小
    edgecolors='black', # 点的边框颜色
    linewidth=0.5       # 边框宽度
)

ax1.set_xlabel('主成分1', fontsize=11)
ax1.set_ylabel('主成分2', fontsize=11)
ax1.set_title('真实标签', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=ax1, label='手势类别')

# 【右图】聚类结果
scatter2 = ax2.scatter(
    X_pca[:, 0],
    X_pca[:, 1],
    c=clusters,         # 颜色：聚类结果（0,1,2）
    cmap='Set1',        # 颜色映射：不同的配色方案
    alpha=0.6,
    s=50,
    edgecolors='black',
    linewidth=0.5
)

# 标记聚类中心
centers_pca = pca.transform(kmeans.cluster_centers_)  # 将聚类中心也降到2维
ax2.scatter(
    centers_pca[:, 0],
    centers_pca[:, 1],
    c='red',            # 中心点用红色
    marker='X',         # X形标记
    s=200,              # 较大的点
    edgecolors='black',
    linewidth=2,
    label='聚类中心'
)

ax2.set_xlabel('主成分1', fontsize=11)
ax2.set_ylabel('主成分2', fontsize=11)
ax2.set_title('K-Means聚类结果', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend()
plt.colorbar(scatter2, ax=ax2, label='簇编号')

plt.tight_layout()
plt.show()

print("\n说明：")
print("  - PCA将18维特征降到2维用于可视化")
print("  - 左图显示真实标签，右图显示聚类结果")
print("  - 如果聚类效果好，两图应该相似（颜色分布一致）")
print("  - 红色X标记是聚类中心（每个簇的中心点）")
print(f"  - 前2个主成分保留了原始数据 {pca.explained_variance_ratio_.sum():.1%} 的信息")

---
## 最终对比总结

In [ ]:
# 详细对比表
comparison = pd.DataFrame([
    {
        '分类器': 'SVM',
        '准确率': f"{acc_svm:.2%}",
        '训练时间': f"{train_time_svm:.4f}s",
        '优点': '高维数据好，泛化强',
        '缺点': '训练慢，参数多',
        '推荐场景': '特征多，数据中等'
    },
    {
        '分类器': 'KNN',
        '准确率': f"{acc_knn:.2%}",
        '训练时间': f"{train_time_knn:.4f}s",
        '优点': '简单直观，无需训练',
        '缺点': '预测慢，内存占用大',
        '推荐场景': '小数据集，快速原型'
    },
    {
        '分类器': 'Random Forest',
        '准确率': f"{acc_rf:.2%}",
        '训练时间': f"{train_time_rf:.4f}s",
        '优点': '准确率高，稳定',
        '缺点': '模型大',
        '推荐场景': '大多数EMG应用（推荐）'
    },
    {
        '分类器': 'MLP神经网络',
        '准确率': f"{acc_mlp:.2%}",
        '训练时间': f"{train_time_mlp:.4f}s",
        '优点': '学习复杂模式',
        '缺点': '需要更多数据',
        '推荐场景': '大数据集，复杂任务'
    }
])

print("\n" + "="*80)
print("分类器详细对比".center(80))
print("="*80)
print(comparison.to_string(index=False))
print("="*80)

### 混淆矩阵对比

In [ ]:
# ===== 绘制四个分类器的混淆矩阵对比 =====

# 什么是混淆矩阵（Confusion Matrix）？
# - 显示模型的预测结果与真实标签的对比
# - 可以看出模型在哪些类别上容易混淆

# --- 第一步：准备所有预测结果 ---
predictions = [
    ('SVM', y_pred_svm),
    ('KNN', y_pred_knn),
    ('Random Forest', y_pred_rf),
    ('MLP', y_pred_mlp)
]

# --- 第二步：创建2×2子图布局 ---
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('混淆矩阵对比', fontsize=16, fontweight='bold')

# 手势名称列表
gesture_names = ['rest', 'fist', 'open']

# --- 第三步：为每个分类器绘制混淆矩阵 ---
for idx, (name, y_pred) in enumerate(predictions):
    # 计算子图位置
    row = idx // 2  # 行号：0或1
    col = idx % 2   # 列号：0或1
    
    # 计算混淆矩阵
    cm = confusion_matrix(y_test, y_pred)
    # 混淆矩阵是一个3×3的矩阵
    # cm[i][j] 表示：真实类别是i，被预测为j的样本数
    # 对角线元素：预测正确的数量
    # 非对角线元素：预测错误的数量
    
    # 使用seaborn绘制热图
    sns.heatmap(
        cm,                     # 混淆矩阵数据
        annot=True,             # 显示数字
        fmt='d',                # 数字格式：整数
        cmap='Blues',           # 颜色映射：蓝色系
        xticklabels=gesture_names,  # x轴标签
        yticklabels=gesture_names,  # y轴标签
        ax=axes[row, col],      # 指定子图
        cbar_kws={'shrink': 0.8}  # 颜色条大小
    )
    
    axes[row, col].set_title(name, fontsize=12, fontweight='bold')
    axes[row, col].set_xlabel('预测标签', fontsize=10)
    axes[row, col].set_ylabel('真实标签', fontsize=10)

plt.tight_layout()
plt.show()

# --- 第四步：混淆矩阵解读说明 ---
print("\n📊 混淆矩阵解读：")
print("=" * 60)
print("混淆矩阵结构：")
print("              预测标签")
print("           rest  fist  open")
print("  真实  rest  [ a    b    c  ]")
print("  标签  fist  [ d    e    f  ]")
print("       open  [ g    h    i  ]")
print()
print("各元素含义：")
print("  - 对角线元素 (a, e, i)：预测正确的数量")
print("    例如：a = 真实是rest，预测也是rest的样本数")
print()
print("  - 非对角线元素：预测错误的数量")
print("    例如：b = 真实是rest，但预测成fist的样本数")
print()
print("理想情况：")
print("  - 对角线元素越大越好（预测正确多）")
print("  - 非对角线元素越小越好（预测错误少）")
print("  - 完美分类器：只有对角线有数字，其他都是0")
print()
print("💡 通过混淆矩阵可以发现：")
print("  - 哪些类别容易被误分类")
print("  - 哪两个类别容易混淆")
print("  - 模型在哪些类别上表现好/差")
print("=" * 60)

---
## 选择建议

### 根据数据量选择
- **小数据集**（< 100样本）：KNN 或 SVM
- **中等数据集**（100-1000样本）：Random Forest（推荐）
- **大数据集**（> 1000样本）：Random Forest 或 MLP

### 根据应用场景选择
- **实时系统**：Random Forest（预测快）
- **离线分析**：任意方法
- **嵌入式设备**：KNN 或简单的决策树
- **研究项目**：对比多种方法

### 推荐的学习顺序
1. 先学**KNN**（最简单，理解分类原理）
2. 再学**Random Forest**（最实用）
3. 然后学**SVM**（理论深入）
4. 最后学**神经网络**（高级应用）